In [19]:
sequences = list()
with open('data/sequences.txt', 'r') as f:
    for line in f:
        sequences.append(line[:-1])

sequences_train = list()
sequences_test = list()
proteins_test = list()
y_train = list()
with open('data/graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            sequences_test.append(sequences[i])
            proteins_test.append(t[0])
        else:
            sequences_train.append(sequences[i])
            y_train.append(int(t[1][:-1]))

len(sequences), len(sequences_train), len(y_train), len(sequences_test), len(proteins_test)

(6111, 4888, 4888, 1223, 1223)

In [20]:
set_char = list(set([letter for sets in map(set, sequences) for letter in sets]))
set_char.sort()
tokenizer = {char:idx for idx,char in enumerate(set_char)}
tok_seq_train = list(map(lambda l: list(map(lambda x : tokenizer[x], list(l))), sequences_train))

import tensorflow as tf
import numpy as np

tfkl = tf.keras.layers

layer = tf.keras.layers.CategoryEncoding(
          num_tokens=len(set_char), output_mode="count")
          
rt = tf.ragged.constant(tok_seq_train)
count_encoding = layer(rt).numpy()


enclabels = tf.keras.layers.CategoryEncoding(
          num_tokens=18, output_mode='one_hot')(tf.expand_dims(y_train, 1)).numpy()

In [21]:
from sklearn.model_selection import train_test_split

#prevents overfitting 
X_train, X_test, y_train, y_test = train_test_split(count_encoding,
                                                    enclabels,
                                                    test_size=0.20,
                                                    random_state=42)

X_tot , y_tot = count_encoding, enclabels

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, dtype=torch.float32), 
    torch.tensor(X_test, dtype=torch.float32), 
    torch.tensor(y_train, dtype=torch.float32), 
    torch.tensor(y_test, dtype=torch.float32)
    )

X_tot, y_tot = torch.tensor(X_tot, dtype=torch.float32), torch.tensor(y_tot, dtype=torch.float32)

In [23]:
from torch.utils.data import DataLoader, Dataset

set_char = list(set([letter for sets in map(set, sequences) for letter in sets]))
set_char.sort()
tokenizer = {char:idx for idx,char in enumerate(set_char)}
tok_seq_train = list(map(lambda l: list(map(lambda x : tokenizer[x], list(l))), sequences_train))

layer = tf.keras.layers.CategoryEncoding(
          num_tokens=len(set_char), output_mode="count")
          
rt = tf.ragged.constant(tok_seq_train)

count_encoding = layer(rt)

# y_train = tf.constant(y_train, dtype=tf.float32)

class ClassifDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence = self.sequences[index]
        label = self.labels[index]

        return sequence, label

dataset = ClassifDataset(
    torch.tensor(X_train), 
    labels=torch.tensor(y_train.numpy()))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/var/folders/fb/c3y45k5d5gdbmk7pnky9y19c0000gn/T/ipykernel_46730/686499914.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X_train),


In [248]:
device = 'cpu'

X_train = X_train.to(device)
y_train = y_train.to(device)

model = nn.Sequential(
    nn.Linear(21, 1024),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(1024, 128),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(128, 128),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(128, 18),
    nn.LogSoftmax(dim=1)
    ).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), 
    # lr=1e-3, 
    lr=1,
    weight_decay=0.01
    )

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1e-8 + (1e-4 - 1e-8) * (1 - epoch / 500))

for epoch in range(500):
    for input, y in dataloader:
        optimizer.zero_grad()
        input, y = input.to(device), y.to(device)
        output = model(input)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

    scheduler.step()
    print(scheduler.get_last_lr()[0])

    train_acc = (torch.argmax(model(X_train), dim=1)==torch.argmax(y_train, dim=1)).to('cpu').numpy().mean()
    print(f"Epoch {epoch}, {100*train_acc}%")

9.980002e-05
Epoch 0, 26.70076726342711%
9.960004e-05
Epoch 1, 34.731457800511514%
9.940006000000001e-05
Epoch 2, 34.168797953964194%
9.920008e-05
Epoch 3, 38.312020460358056%
9.90001e-05
Epoch 4, 39.923273657289%
9.880012e-05
Epoch 5, 41.66240409207161%
9.860014000000001e-05
Epoch 6, 43.09462915601023%
9.840016e-05
Epoch 7, 43.32480818414322%
9.820018e-05
Epoch 8, 43.22250639386189%
9.80002e-05
Epoch 9, 42.86445012787724%
9.780022000000001e-05
Epoch 10, 45.549872122762146%
9.760024e-05
Epoch 11, 44.37340153452685%
9.740026e-05
Epoch 12, 44.75703324808184%
9.720028e-05
Epoch 13, 45.44757033248082%
9.700030000000001e-05
Epoch 14, 46.265984654731454%
9.680032e-05
Epoch 15, 46.44501278772378%
9.660034e-05
Epoch 16, 47.084398976982094%
9.640036e-05
Epoch 17, 47.77493606138108%
9.620038e-05
Epoch 18, 46.08695652173913%
9.60004e-05
Epoch 19, 47.8772378516624%
9.580042e-05
Epoch 20, 47.13554987212276%
9.560044e-05
Epoch 21, 47.928388746803066%
9.540046e-05
Epoch 22, 48.107416879795394%
9.5200

In [249]:
model.eval()
model.training

False

In [250]:
train_acc = (torch.argmax(model(X_train), dim=1)==torch.argmax(y_train, dim=1)).numpy().mean()
test_acc = (torch.argmax(model(X_test), dim=1)==torch.argmax(y_test, dim=1)).numpy().mean()
train_acc, test_acc

(0.7033248081841432, 0.5480572597137015)

In [251]:
from sklearn.metrics import accuracy_score, log_loss

log_loss(y_true = y_test, 
        y_pred = torch.nn.Softmax(dim=1)(model(X_test)).detach().numpy())

1.5270320462031306

1.550464783515805 avec 1024 > 128 > 128 > 18 dropout a 0.3 et 2*0.2

1.5220981442226547
